In [2]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import json, base64, time, random

def aes_encrypt(plain_text: str, key: bytes, iv: bytes) -> str:
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ct_bytes = cipher.encrypt(pad(plain_text.encode('utf-8'), AES.block_size))
    return base64.b64encode(iv + ct_bytes).decode('utf-8')

def aes_decrypt(b64_cipher: str, key: bytes) -> str:
    data = base64.b64decode(b64_cipher)
    iv = data[:16]
    ct = data[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), AES.block_size)
    return pt.decode('utf-8')

# خلية: توليد قراءات عشوائية وتشفيرها
def generate_sensor_reading():
    temp = round(random.uniform(18.0, 40.0), 2)       # درجة حرارة
    hum  = round(random.uniform(20.0, 90.0), 2)       # رطوبة
    ts   = int(time.time())
    return {'timestamp': ts, 'temperature': temp, 'humidity': hum}

# مفتاح AES 128-bit (في جهاز حقيقي يجب حفظه بأمان / HSM)
key = get_random_bytes(16)

# محاكاة جهاز (Device) يرسل بيانات مشفَّرة إلى الخادم (Server)
device_payload = generate_sensor_reading()
device_payload_json = json.dumps(device_payload)
iv = get_random_bytes(16)
ciphertext_b64 = aes_encrypt(device_payload_json, key, iv)

print("=== Device (قبل التشفير) ===")
print(device_payload_json)
print("\n=== المرسل (مشفر base64) ===")
print(ciphertext_b64)

# خلية: خادم يستقبل ويفك تشفير
received = ciphertext_b64  # محاكاة النقل
decrypted_json = aes_decrypt(received, key)
print("\n=== المستلم (بعد فك التشفير) ===")
print(decrypted_json)

=== Device (قبل التشفير) ===
{"timestamp": 1764431351, "temperature": 25.04, "humidity": 46.0}

=== المرسل (مشفر base64) ===
npmgFPXtw51Hc99SlXCURtg75zQGHDtP4893uqtm57dqbLaFDHb+5iqBv4hruWHNCI0ivrl1T0zsLE/tVlSqeySmzB6Byae84b3A0uNCNlsKK2yEuOgzVlQB5Mp4+5z3

=== المستلم (بعد فك التشفير) ===
{"timestamp": 1764431351, "temperature": 25.04, "humidity": 46.0}
